# Get target video list from target uesr list

In [69]:
import os
import glob
import json
import csv
import re

# input: 'targetusr.csv', 'videodata.json'
# output: 'targetvideo.csv' - list of video ids that target users studied. 
curdir=os.getcwd()

targetuserfile='targetuser.csv'

with open(targetuserfile,'r',encoding='utf-8') as uf:
    ureader=csv.reader(uf)
    targetusers=[]
    next(ureader)
    for row in ureader:
        targetusers.append(row[0])
    uf.close()
    
data=json.load(open('videodata.json','r',encoding='utf-8'))

targetvideoset=set()
for event in data:
    vid=event["video_id"]
    uid=event["user_id"]
    if(uid in targetusers):
        targetvideoset.add(vid)

targetvideofile='targetvideos.csv'
    
with open(targetvideofile,'w',encoding='utf-8') as vf:
    vwriter=csv.writer(vf,delimiter=',')
    for vid in targetvideoset:
        vwriter.writerow([vid])
    vf.close()
  


# Filter out non-target events

In [70]:
targetdata=[]
for vid in targetvideoset:
    newobj={}
    newobj["video_id"]=vid 
    newobj["events"]=[]
    targetdata.append(newobj)

for event in data:
    videoid=event["video_id"]
    for vobj in targetdata:
        vid=vobj["video_id"]
        if(vid==event["video_id"]):
            vobj["course_id"]=event['course_id']
            vobj["org_id"]=event['org_id']
            vobj["week_id"]=event['week_id']
            vobj["events"].append(event)
                  
with open('video_targetdata.json','w',encoding='utf-8') as wf:
    eventwriter=csv.writer(wf,delimiter=',')
    json.dump(targetdata,wf, sort_keys=False, indent=4)
    wf.close()

# Generate event data with video, session info 

In [71]:
def extract_time(event):
    return event['time']


# restructure as session-wise data
newtargetdata=[]
for vid in targetvideoset:
    for vobj in targetdata:
        if(vobj["video_id"]==vid):
            newvobj={}
            newvobj["video_id"]=vid
            newvobj["course_id"]=vobj["course_id"]
            newvobj["org_id"]=vobj["org_id"]
            newvobj["week_id"]=vobj["week_id"]            
            events=vobj["events"]
            sessionset=set() # this gives unique list of sessions related to this vid 
            for event in events:
                sessionid=event["session"]
                if(sessionid):
                    sessionset.add(sessionid)
            sessionevent=[]            
            
            for session in sessionset:
                sessiondata={}
                sessiondata["session"]=session
                sessiondata["events"]=[]
                sessionevent.append(sessiondata)
                
#            print(sessionevent[0])

            for event in events:
                sessionid=event["session"]
                for  s in sessionevent:
                    if s['session'] == sessionid:
                        s['user_id']=event['user_id']
                        del event["course_id"]
                        del event["org_id"]
                        del event["session"]
                        del event["user_id"]
                        del event["video_id"]
                        del event["week_id"]
                        s['events'].append(event)
                        s['events']=sorted(s['events'], key=lambda k: k.get('time',0), reverse=False)
                        break
                    else:
                        continue
                    print('Nothing found!')
            newvobj["sessions"]=sessionevent
    newtargetdata.append(newvobj)

with open('video_targetdata_session.json','w',encoding='utf-8') as wf:
    eventwriter=csv.writer(wf,delimiter=',')
    json.dump(newtargetdata,wf, sort_keys=False, indent=4)
    wf.close()


# Extend to contain pause duration

In [72]:
import datetime
import dateutil.parser

dt1="2018-07-24T15:34:11.481975+00:00"
dt2="2018-07-24T15:36:22.311502+00:00"

odt1=dateutil.parser.parse(dt1)
odt2=dateutil.parser.parse(dt2)

print(odt1)
print(odt2)
print(odt2-odt1)


2018-07-24 15:34:11.481975+00:00
2018-07-24 15:36:22.311502+00:00
0:02:10.829527


In [73]:
oldtargetdata=json.load(open('video_targetdata_session.json','r',encoding='utf-8'))

newtargetdata=[]
for vobj in oldtargetdata:
    newvobj=vobj
    newsessions=[]
    for s in vobj["sessions"]:
        events=s["events"]
        news=s
        newevents=[]
        prevet="play"
        for i in range(len(events)):
            event=events[i]
            newevents.append(event)
            et=event["event_type"]
            if(et!="stop_video"):
                if(prevet=="pause_video"):
                    curtime=dateutil.parser.parse(event["time"])
                    pausetime=dateutil.parser.parse(events[i-1]["time"])
                    pauseduration=curtime-pausetime
                    newevents[i-1]["duration"]=pauseduration.total_seconds()
  #                  print("Hi")
            prevet=et
        news["events"]=newevents
 #       print(news)
        newsessions.append(news)
    newvobj["sessions"]=newsessions
    newtargetdata.append(newvobj)

with open('video_targetdata_session_with_duration.json','w',encoding='utf-8') as wf:
    eventwriter=csv.writer(wf,delimiter=',')
    json.dump(newtargetdata,wf, sort_keys=False, indent=4)
    wf.close()
           
            
            